In [ ]:
import requests
from autogen import ConversableAgent, register_function

In [ ]:
#TODO: future config_file should be a json file to not repeat this lines
config_list = [{
        "model": "llama3.1",
        "base_url": "http://localhost:11434/v1",
        "api_key": "ollama"
}]

In [ ]:

def describe_country(country_name:str)->str:
    url = "https://en.wikipedia.org/w/api.php?format=json&action=query&prop=extracts&exintro&explaintext&redirects=1&titles="+country_name
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check if the request was successful
        json_data = response.json()  # Extract JSON content
        page = next(iter(json_data["query"]["pages"].values()))
        return page["extract"]
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except Exception as err:
        print(f"Other error occurred: {err}")



In [ ]:
wiki_tool = ConversableAgent(
    "wiki_tool",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config=False,
    # is_termination_msg=lambda msg: "terminate" in msg["content"].lower(),
)

wiki_tool_driver = ConversableAgent(
    "wiki_tool_driver",
    system_message = "You are a helpful assistant. You may decide to use some external tools if necessary"
    "If you are happy with the result of the tool, just say TERMINATE",
    llm_config = {"config_list": config_list},
    code_execution_config=False
)

register_function(
    describe_country,
    caller = wiki_tool_driver,
    executor = wiki_tool,
    description = "get the description of a country from wiki. this service only receive the formal name of a country"
)

wiki_tool.initiate_chat(
    wiki_tool_driver,
    message = "I want to know something about a country with nick name uncle sam",
    max_turns = 2
)